In [1]:
import requests
api_key = "61f6365b-e83b-491b-9bcd-06854cdcbcae"
url = "https://api.harvardartmuseums.org/classification"

Parameter = {
    "apikey":api_key,
    "size":100
}

res = requests.get(url,Parameter)
res

<Response [200]>

In [2]:
data = res.json()
data

{'info': {'totalrecordsperquery': 100,
  'totalrecords': 64,
  'pages': 1,
  'page': 1,
  'responsetime': '21 ms'},
 'records': [{'objectcount': 1,
   'name': 'Natural History Specimens',
   'id': 1133,
   'lastupdate': '2025-08-19T04:34:33-04:00',
   'classificationid': 1133},
  {'objectcount': 1,
   'name': 'Paintings with Text',
   'id': 197,
   'lastupdate': '2025-08-19T04:34:33-04:00',
   'classificationid': 197},
  {'objectcount': 66,
   'name': 'Performance Artifacts',
   'id': 224,
   'lastupdate': '2025-08-19T04:34:33-04:00',
   'classificationid': 224},
  {'objectcount': 470,
   'name': 'Rubbings',
   'id': 173,
   'lastupdate': '2025-08-19T04:34:33-04:00',
   'classificationid': 173},
  {'objectcount': 62,
   'name': 'Brick Stamps',
   'id': 304,
   'lastupdate': '2025-08-19T04:34:33-04:00',
   'classificationid': 304},
  {'objectcount': 46,
   'name': 'Cameos',
   'id': 1086,
   'lastupdate': '2025-08-19T04:34:33-04:00',
   'classificationid': 1086},
  {'objectcount': 1,
  

In [3]:
# Extract names with objectcount > 2500
for record in data["records"]:
    if record["objectcount"]>2500:
        print(record["name"])

Accessories (non-art)
Photographs
Drawings
Prints
Paintings
Sculpture
Coins
Vessels
Textile Arts
Archival Material
Fragments
Manuscripts
Seals
Straus Materials


In [1]:
import requests
import pandas as pd

api_key = "61f6365b-e83b-491b-9bcd-06854cdcbcae"
url = "https://api.harvardartmuseums.org/object"
classifications = ['Paintings']

all_records = []

for cls in classifications:
    print(f"{cls}")
    for page in range(1, 26):
        print(f"  Page {page}", end=" ")
        params = {
            'apikey': api_key,
            'classification': cls,
            'size': 100,
            'page': page
        }
        try:
            res = requests.get(url, params=params)
            data = res.json()
            records = data.get('records', [])
            if not records:
                print("No more data.")
                break
            all_records.extend(records)
            print("Done")
        except:
            print("Error occurred.")
            break

if all_records:
    pd.DataFrame(all_records).to_csv('harvard_art_data.csv', index=False)
    print("\n Data saved to harvard_art_data.csv")
else:
    print("\n No data was collected.")


Paintings
  Page 1 Done
  Page 2 Done
  Page 3 Done
  Page 4 Done
  Page 5 Done
  Page 6 Done
  Page 7 Done
  Page 8 Done
  Page 9 Done
  Page 10 Done
  Page 11 Done
  Page 12 Done
  Page 13 Done
  Page 14 Done
  Page 15 Done
  Page 16 Done
  Page 17 Done
  Page 18 Done
  Page 19 Done
  Page 20 Done
  Page 21 Done
  Page 22 Done
  Page 23 Done
  Page 24 Done
  Page 25 Done

 Data saved to harvard_art_data.csv


In [2]:
records = []

for i in data['records']:
    records.append({
        "id": i.get("id"),
        "title": i.get("title"),
        "culture": i.get("culture"),
        "period": i.get("period"),
        "century": i.get("century"),
        "medium": i.get("medium"),
        "dimensions": i.get("dimensions"),
        "description": i.get("description"),
        "department": i.get("department"),
        "classification": i.get("classification"),
        "accessionyear": i.get("accessionyear"),
        "accessionmethod":i.get("accessionmethod")
    })


In [3]:
media_records = []

for i in data['records']:
    media_records.append({
        "objectid": i.get("id"),
        "imagecount": i.get("imagecount"),
        "mediacount": i.get("mediacount"),
        "colorcount": i.get("colorcount"),
        "rank": i.get("rank"),
        "datebegin": i.get("datebegin"),
        "dateend": i.get("dateend")
    })


In [ ]:
colors = []

for i in data['records']:
    for c in i.get("colors", []):
        colors.append({
            'objectid': i.get('id'),          
            'color': c.get('color'),
            'spectrum': c.get('spectrum'),
            'hue': c.get('hue'),
            'percent': c.get('percent'),
            'css': c.get('css3')
        })


In [54]:
import pymysql

connection = pymysql.connect(
    host="localhost",
    user="root",
    password="Azhagu1910@",
    database="Harvard_Artifactse"
)
cursor = connection.cursor()

#  Create artifact_metadata
cursor.execute ("""
    CREATE TABLE IF NOT EXISTS artifact_metadata (
        id INT PRIMARY KEY,
        title TEXT,
        culture TEXT,
        period TEXT,
        century TEXT,
        medium TEXT,
        dimensions TEXT,
        description TEXT,
        department TEXT,
        classification TEXT,
        accessionyear INT,
        accessionmethod TEXT
    )
""")

#  Create artifact_media
cursor.execute("""
    CREATE TABLE IF NOT EXISTS artifact_media (
        objectid INT,
        imagecount INT,
        mediacount INT,
        colorcount INT,
        `rank` INT,
        datebegin INT,
        dateend INT,
        FOREIGN KEY (objectid) REFERENCES artifact_metadata(id)
    )
""")


#  Create artifact_colors
cursor.execute("""
    CREATE TABLE IF NOT EXISTS artifact_colors (
        objectid INT,
        color TEXT,
        spectrum TEXT,
        hue TEXT,
        percent REAL,
        css TEXT,
        FOREIGN KEY (objectid) REFERENCES artifact_metadata(id)
    )
""")
# Commit and close
connection.commit()
cursor.close()
connection.close()

print("Data inserted into tables.")




Data inserted into tables.


In [55]:
import pymysql

# Step: Connect to DB (repeat this only if not already connected above)
connection = pymysql.connect(
    host="localhost",
    user="root",
    password="Azhagu1910@",
    database="Harvard_Artifactse"
)
cursor = connection.cursor()

for i in records:
    cursor.execute ("""
        INSERT IGNORE INTO artifact_metadata (
            id, title, culture, period, century, medium, dimensions,
            description, department, classification, accessionyear, accessionmethod
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        i.get("id"), i.get("title"), i.get("culture"), i.get("period"),
        i.get("century"), i.get("medium"), i.get("dimensions"), i.get("description"),
        i.get("department"), i.get("classification"), i.get("accessionyear"),
        i.get("accessionmethod")
    ))


In [56]:
for m in media_records:
    cursor.execute("""
        INSERT IGNORE INTO artifact_media (
            objectid, imagecount, mediacount, colorcount, `rank`, datebegin, dateend
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        m.get("objectid"),   
        m.get("imagecount"),
        m.get("mediacount"),
        m.get("colorcount"),
        m.get("rank"),
        m.get("datebegin"),
        m.get("dateend")
    ))


In [57]:
for j in colors:
    cursor.execute("""
        INSERT IGNORE INTO artifact_colors (
            objectid, color, spectrum, hue, percent, css
        ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        j.get("objectid"),
        j.get("color"),
        j.get("spectrum"),
        j.get("hue"),
        j.get("percent"),
        j.get("css")
    ))


In [58]:
# Commit and close
connection.commit()
cursor.close()
connection.close()

print("Data inserted into tables.")

Data inserted into tables.
